In [3]:
from config import api_key
import requests
 
api_key = api_key
header = {"X-Riot-Token" : api_key}
 
def league_v4_tier_info(tier):
    queue = "RANKED_SOLO_5x5"
    url = f"https://kr.api.riotgames.com/lol/league/v4/{tier}leagues/by-queue/{queue}"
    requests.get(url, headers=header)
    response = requests.get(url, headers=header)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data: {response.status_code}")
        return None

In [5]:
master_plus = ["challenger", "grandmaster", "master"]
master_plus_play = {}
player_json = []

for tier in master_plus:
    tier_data = league_v4_tier_info(tier)
    if tier_data:
        for entry in tier_data['entries']:
            entry['tier'] = tier.capitalize()  # 티어 정보를 추가
        player_json.extend(tier_data['entries'])

sorted_entries = sorted(player_json, key=lambda x: x['leaguePoints'], reverse=True)
for index, entry in enumerate(sorted_entries, start=1):
    master_plus_play[index] = {
        "Name": entry["summonerId"],
        "Rank": entry["tier"] + entry["rank"], 
        "LP": entry["leaguePoints"],
        "wins": entry["wins"],
        "losses": entry["losses"]
    }

print(master_plus_play)
print(master_plus_play[12])

{1: {'Name': 'G1OjDZapvgrRJlHXEmz-I3HIptaSH3tohWHgz1rz2Wx_3g0', 'Rank': 'ChallengerI', 'LP': 1503, 'wins': 118, 'losses': 76}, 2: {'Name': 'or5--CL3dbKZlMz3Lh-kDstKM1WOFG0eKJDZrUPFoeRnvsA', 'Rank': 'ChallengerI', 'LP': 1405, 'wins': 114, 'losses': 88}, 3: {'Name': 'dF9F3NRdYNBKrIwOMMAX3lYTeMOmvM5hRInM5UAALO4DL2U', 'Rank': 'ChallengerI', 'LP': 1350, 'wins': 88, 'losses': 56}, 4: {'Name': '2IlOdq5IIUgX1rmIFZOuUiQo4ErEEXkoFPuAJEnAndRHX0c', 'Rank': 'ChallengerI', 'LP': 1305, 'wins': 92, 'losses': 84}, 5: {'Name': 'z2rfkrJEVn1W4ggdVmJ_4K3ORktgOxwT03N_dZs1NI4f5anACI8rkURkHA', 'Rank': 'ChallengerI', 'LP': 1300, 'wins': 87, 'losses': 57}, 6: {'Name': 'UzIEYIRdhYP2U5gSPVThdaXrAzxdJBwHEMnqasPLx8-RuNCGB8b_tlGSvQ', 'Rank': 'ChallengerI', 'LP': 1255, 'wins': 77, 'losses': 55}, 7: {'Name': '_zxy66Lpu3hMs7_Ga5XXNKn_C1OCgF8xRYoQ0LFBg7O7bKd_TJlsHeME_Q', 'Rank': 'ChallengerI', 'LP': 1195, 'wins': 92, 'losses': 74}, 8: {'Name': 'zgUoMtda1LlmLAfTJBML2ZPaQ7BmZCYuQ_270t-jrdOMbTT74IEHR3UlJQ', 'Rank': 'Challe